# 필요한 라이브러리 import

In [10]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf

warnings.filterwarnings(action='ignore')

In [11]:
with open('./models/forward_ss.pickle','rb') as f:
    forward_ss = pickle.load(f)
    
with open('./models/forward_qt.pickle','rb') as f:
    forward_qt = pickle.load(f)

forward_model = load_model('./models/forward_nn.h5')

    
premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [12]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [13]:
forward_df = premier_df[premier_df['포지션']=='공격수']

In [14]:
use_col = ['골','도움','크로스 성공률','파울 획득','반칙 수/90', '기회 창출/90', '경기당 유효 슈팅/90', '경기당 슈팅/90', '인터셉트/90',
       '달린 거리/90분', '공중볼시도/90', '태클 성공률', '경기당 패스 시도/90', '경기당 패스 성공/90',
       '키패스/90', '태클 성공/90', '드리블 성공/90', '헤더 성공/90', '크로스 시도/90','평균획득승점/경기']

In [15]:
outlier_features = ['크로스 성공률', '반칙 수/90', '파울 획득', '골', '도움', '헤더 성공/90', '기회 창출/90', '태클 성공/90']
nonoutlier_features = ['공중볼시도/90','경기당 패스 성공/90','달린 거리/90분','경기당 유효 슈팅/90', '드리블 성공/90', '키패스/90',
                       '인터셉트/90', '태클 성공률', '크로스 시도/90', '경기당 슈팅/90', '경기당 패스 시도/90']

In [16]:
forward_df.loc[:,outlier_features] = forward_qt.transform(forward_df.loc[:,outlier_features])
forward_df.loc[:,nonoutlier_features] = forward_ss.transform(forward_df.loc[:,nonoutlier_features])

In [19]:
forward_df = forward_df.reset_index(drop=True)

In [17]:
good_features = ['골', '도움', '경기당 유효 슈팅/90', '공중볼시도/90']

In [20]:
forward_df['예측PPG']=forward_model.predict(tf.constant(forward_df.loc[:,good_features].values))
forward_df

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,-0.248378,-0.069755,0,7,11,...,1.907102,-0.565198,-0.613374,-1.104166,-0.500413,-0.656151,-1.496670,-0.998534,0.94,1.594121
1,Milot Rashica,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,-0.944738,1.131520,0,8,9,...,0.615068,0.273872,0.202855,0.819425,0.240543,0.738278,-1.545002,2.187099,1.03,1.482862
2,Christos Tzolis,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,-2.009450,-5.199338,0,7,7,...,0.061339,0.055398,-0.096429,0.694516,0.976908,-0.203259,1.042306,0.329184,1.04,1.069338
3,Callum Wilson,2021-2022,공격수,뉴캐슬,잉글랜드 프리미어 리그,1.198980,-0.069755,0,20,5,...,-0.584678,-1.448597,-1.694878,-0.479623,-0.696126,-0.811087,0.001267,-0.757941,1.81,1.676256
4,Allan Saint-Maximin,2021-2022,공격수,뉴캐슬,잉글랜드 프리미어 리그,-0.944738,1.131520,0,22,6,...,0.522780,-0.790006,-0.810629,0.969315,1.055486,1.215006,-1.382445,0.436115,1.76,1.483803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Odsonne Edouard,2021-2022,공격수,팰리스,잉글랜드 프리미어 리그,1.606382,-0.573615,0,15,11,...,-0.953831,-0.979984,-0.967073,-0.854349,-1.103132,-0.989860,0.133457,-0.766852,1.27,1.634921
63,Wilfried Zaha,2021-2022,공격수,팰리스,잉글랜드 프리미어 리그,0.106651,1.131520,0,14,9,...,0.338204,1.340917,1.376184,0.519645,0.211940,1.524879,-1.639485,0.231165,1.34,1.512454
64,Harry Kane,2021-2022,공격수,토트넘,잉글랜드 프리미어 리그,1.955228,2.179123,0,25,16,...,-1.507560,-0.865997,-0.885450,0.119937,-0.883162,-1.001778,0.454956,-0.833683,1.82,1.948370
65,Heung-Min Son,2021-2022,공격수,토트넘,잉글랜드 프리미어 리그,1.606382,1.131520,0,22,17,...,-0.215526,0.964127,1.093905,-0.329733,-0.040563,1.048151,-0.944738,0.378194,1.66,1.958791


In [21]:
forward_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG
58,Romelu Lukaku,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,5.199338,-5.199338,0,36,10,...,-0.769255,-0.698183,-0.671190,-1.104166,-1.695451,-0.453541,0.338368,-0.909425,2.03,2.279693
12,Mohamed Salah,2021-2022,공격수,리버풀,잉글랜드 프리미어 리그,1.385747,1.520392,0,32,13,...,0.430492,0.976792,0.726602,1.493931,0.078651,0.809787,-0.930967,1.541062,1.98,2.214604
60,Kai Havertz,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,1.734034,1.520392,0,34,11,...,0.338204,2.658099,2.546112,1.593858,0.147535,0.142368,-0.582601,-0.316853,2.05,2.140351
22,Cristiano Ronaldo,2021-2022,공격수,맨 레드,잉글랜드 프리미어 리그,1.606382,1.706217,0,22,13,...,-0.676966,-0.296062,-0.293684,0.844407,-0.040563,-0.012568,0.326315,-0.160913,1.80,2.088289
65,Heung-Min Son,2021-2022,공격수,토트넘,잉글랜드 프리미어 리그,1.606382,1.131520,0,22,17,...,-0.215526,0.964127,1.093905,-0.329733,-0.040563,1.048151,-0.944738,0.378194,1.66,1.958791


In [24]:
forward_min = np.min(forward_df['예측PPG'].tolist())
forward_min

0.9656938910484314

In [25]:
forward_df['WAR'] = forward_df['예측PPG'].map(lambda x : x-forward_min+1)

In [27]:
forward_df.sort_values(by='WAR',ascending=False).head(60)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG,WAR
58,Romelu Lukaku,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,5.199338,-5.199338,0,36,10,...,-0.698183,-0.671190,-1.104166,-1.695451,-0.453541,0.338368,-0.909425,2.03,2.279693,2.313999
12,Mohamed Salah,2021-2022,공격수,리버풀,잉글랜드 프리미어 리그,1.385747,1.520392,0,32,13,...,0.976792,0.726602,1.493931,0.078651,0.809787,-0.930967,1.541062,1.98,2.214604,2.248910
60,Kai Havertz,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,1.734034,1.520392,0,34,11,...,2.658099,2.546112,1.593858,0.147535,0.142368,-0.582601,-0.316853,2.05,2.140351,2.174657
22,Cristiano Ronaldo,2021-2022,공격수,맨 레드,잉글랜드 프리미어 리그,1.606382,1.706217,0,22,13,...,-0.296062,-0.293684,0.844407,-0.040563,-0.012568,0.326315,-0.160913,1.80,2.088289,2.122596
65,Heung-Min Son,2021-2022,공격수,토트넘,잉글랜드 프리미어 리그,1.606382,1.131520,0,22,17,...,0.964127,1.093905,-0.329733,-0.040563,1.048151,-0.944738,0.378194,1.66,1.958791,1.993098
64,Harry Kane,2021-2022,공격수,토트넘,잉글랜드 프리미어 리그,1.955228,2.179123,0,25,16,...,-0.865997,-0.885450,0.119937,-0.883162,-1.001778,0.454956,-0.833683,1.82,1.948370,1.982676
23,Jadon Sancho,2021-2022,공격수,맨 레드,잉글랜드 프리미어 리그,0.936847,2.479828,0,23,14,...,1.540394,1.379585,0.969315,0.184798,1.798997,-0.800087,0.935123,1.63,1.946402,1.980708
18,Phil Foden,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,-0.073567,0.921238,0,21,2,...,3.224868,3.267115,1.369022,0.950695,0.809787,-1.861387,0.779183,2.50,1.884626,1.918932
19,Riyad Mahrez,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,-0.248378,1.334616,0,20,3,...,1.432740,1.437401,2.318327,0.718914,1.250760,-1.938368,1.402943,2.42,1.883433,1.917740
39,Ollie Watkins,2021-2022,공격수,아스톤 빌라,잉글랜드 프리미어 리그,1.080138,1.520392,0,13,8,...,-1.030645,-0.929663,-0.279770,-0.480404,-0.620396,-0.840899,-0.410417,1.07,1.868453,1.902759


In [28]:
forward_df.query(f'이름 in ["Phil Foden"]')[['이름','WAR']]

,이름,WAR
18,Phil Foden,1.918932


In [29]:
forward_df.loc[:,outlier_features] = forward_qt.inverse_transform(forward_df.loc[:,outlier_features])
forward_df.loc[:,nonoutlier_features] = forward_ss.inverse_transform(forward_df.loc[:,nonoutlier_features])

In [33]:
forward_df[['이름','골', '도움', '경기당 유효 슈팅/90', '공중볼시도/90','WAR']].sort_values(by='WAR',ascending=False).head(10)

,이름,골,도움,경기당 유효 슈팅/90,공중볼시도/90,WAR
58,Romelu Lukaku,36.0,0.0,2.28,11.00,2.313999
12,Mohamed Salah,19.0,8.0,1.71,6.18,2.248910
60,Kai Havertz,22.0,8.0,1.44,6.66,2.174657
22,Cristiano Ronaldo,21.0,9.0,1.95,8.86,2.122596
65,Heung-Min Son,21.0,6.0,1.48,8.10,1.993098
64,Harry Kane,25.0,12.0,2.01,12.64,1.982676
23,Jadon Sancho,15.0,14.0,0.97,6.02,1.980708
18,Phil Foden,8.0,5.0,1.26,4.57,1.918932
19,Riyad Mahrez,7.0,7.0,1.29,4.80,1.917740
39,Ollie Watkins,16.0,8.0,1.37,7.78,1.902759
